In [ ]:
"""

Prepare sub-shelf melt forcing to be used in climate models?

"""

In [1]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm.notebook import tqdm

In [2]:
%matplotlib qt5

QStandardPaths: error creating runtime directory '/run/user/2784' (Permission denied)


In [3]:
inputpath_raw = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/raw/'
inputpath_interim = '/data/cburgard/PREPARE_FORCING/PREPARE_CAVITY_MASKS/interim/'
inputpath_Schmidt = '/data/cburgard/PREPARE_FORCING/PREPARE_PRESCRIBED_MELT/Schmidt_stuff/'

In [4]:
BedMachine_05 = xr.open_dataset(inputpath_Schmidt + 'BedMachine3_05degrees.nc')
BedMachine_05_mask = xr.open_dataset(inputpath_Schmidt + 'BedMachine3_mask_05degrees.nc')

In [5]:
BedMachine_05_mask_clean = BedMachine_05_mask['mask'].where(BedMachine_05_mask['mask'] != 4,2).where(BedMachine_05_mask['mask']!= 1,2)

In [6]:
file_Justine_orig = xr.open_dataset(inputpath_raw + 'Mask_Iceshelf_IMBIE2_v2.nc')
mask_Justine_05 = xr.open_dataset(inputpath_Schmidt + 'Iceshelf_extrap_05degrees.nc')

In [7]:
file_Justine_withzmin_zmax = xr.open_dataset(inputpath_interim + 'Mask_Iceshelf_IMBIE2_v2_with_zmin_zmax_isfconc.nc')


In [28]:
file_Justine_withzmin_zmax

<xarray.Dataset>
Dimensions:                  (ID: 133, x: 3041, y: 3041)
Coordinates:
  * ID                       (ID) int64 1 2 3 4 5 6 ... 128 129 130 131 132 133
  * x                        (x) int32 -3040000 -3038000 ... 3038000 3040000
  * y                        (y) int32 -3040000 -3038000 ... 3038000 3040000
    lat                      (y, x) float64 ...
    lon                      (y, x) float64 ...
    quantile                 float64 ...
Data variables: (12/21)
    NAME                     (ID) object ...
    Regions                  (ID) object ...
    Subregions               (ID) object ...
    MeltRignot               (ID) float64 ...
    UncertaintiesRignot      (ID) float64 ...
    Iceshelf                 (y, x) float32 ...
    ...                       ...
    UncertaintiesDavison     (ID) float64 ...
    z_min                    (ID) float64 ...
    z_max                    (ID) float64 ...
    z_perc01                 (ID) float64 ...
    z_perc99                 (ID) float64 ...
    isf_conc                 (y, x) float64 ...

In [9]:
ID_list = {}
for bb in file_Justine_orig['Subregions'].values:
    ID_list[bb] = []
    for kisf in file_Justine_orig['ID'].values:
        if file_Justine_orig['Subregions'].sel(ID=kisf) == bb:
            ID_list[bb].append(kisf)

totmelt_list = []
zmin_list = []
zmax_list = []

melt_src = 'Davison' # options: 'Adusumilli','Rignot', 'Paolo', 'Davison'
for bb in ID_list.keys():
    # Compute total melt per basin
    melt_basin = file_Justine_withzmin_zmax['Melt'+melt_src].sel(ID=ID_list[bb]).sum('ID')
    totmelt_list.append(melt_basin.assign_coords({'basin': bb}))
    
    # identify minimum zmin and maximum zmax in each basin
    zmin_basin = file_Justine_withzmin_zmax['z_min'].sel(ID=ID_list[bb]).min('ID')
    zmax_basin = file_Justine_withzmin_zmax['z_max'].sel(ID=ID_list[bb]).max('ID')
    zmin_list.append(zmin_basin.assign_coords({'basin': bb}))
    zmax_list.append(zmax_basin.assign_coords({'basin': bb}))

melt_all = xr.concat(totmelt_list, dim='basin')
zmin_all = xr.concat(zmin_list, dim='basin')
zmax_all = xr.concat(zmax_list, dim='basin')

In [10]:
# with quantiles
totmelt_list = []
zmin_list = []
zmax_list = []

melt_src = 'Davison' # options: 'Adusumilli','Rignot', 'Paolo', 'Davison'
for bb in ID_list.keys():
    # Compute total melt per basin
    melt_basin = file_Justine_withzmin_zmax['Melt'+melt_src].sel(ID=ID_list[bb]).sum('ID')
    totmelt_list.append(melt_basin.assign_coords({'basin': bb}))
    
    # identify minimum zmin and maximum zmax in each basin
    zmin_basin = file_Justine_withzmin_zmax['z_perc01'].sel(ID=ID_list[bb]).min('ID')
    zmax_basin = file_Justine_withzmin_zmax['z_perc99'].sel(ID=ID_list[bb]).max('ID')
    zmin_list.append(zmin_basin.assign_coords({'basin': bb}))
    zmax_list.append(zmax_basin.assign_coords({'basin': bb}))

melt_all = xr.concat(totmelt_list, dim='basin')
zmin_all = xr.concat(zmin_list, dim='basin')
zmax_all = xr.concat(zmax_list, dim='basin')

In [11]:
bathy = BedMachine_05['bed']
isf_draft = BedMachine_05['thickness'] - BedMachine_05['surface']
isf_draft = isf_draft.where(isf_draft > 0,0).where(BedMachine_05_mask_clean == 3,0)

In [12]:
# Create a mask discriminating between land, ocean and ice shelf
mask_0_1_2 = BedMachine_05_mask_clean.copy()
mask_0_1_2 = mask_0_1_2.where(BedMachine_05_mask_clean != 2, 400) # land
mask_0_1_2 = mask_0_1_2.where(BedMachine_05_mask_clean != 0, 0) # ocean
mask_0_1_2 = mask_0_1_2.where(BedMachine_05_mask_clean != 3, 200) # ice shelf

In [13]:
# set all ice shelves to 300
mask_front0 = mask_0_1_2.where((mask_0_1_2 == 0) | (mask_0_1_2 == 400), 300).copy()

In [14]:
# Instead of ice shelves, define IMBIE domains
extrap_domains = mask_Justine_05['Iceshelf_extrap']
IMBIE_domains = extrap_domains.copy()
for bnumb,bb in enumerate(ID_list.keys()):
    #print(bnumb+1)
    mask_dom = xr.DataArray(data=np.isin(extrap_domains, np.array(ID_list[bb])), dims=extrap_domains.dims)
    IMBIE_domains = IMBIE_domains.where(~mask_dom, bnumb+1)

In [15]:
mask_front = mask_front0.copy()
# check all directions and set points at border between ocean and ice shelf (300-0) to 500
mask_front = mask_front.where((mask_front0.shift(lon=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(lon=1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(lat=-1)-mask_front0)!=300,500)
mask_front = mask_front.where((mask_front0.shift(lat=1)-mask_front0)!=300,500)
# cut out all front points
mask_front = mask_front.where(mask_front==500)
# set the ice shelf number
mask_front = mask_front.where(mask_front!=500,IMBIE_domains)

In [24]:
mask_front.plot()

Prescribe Davison melt

In [16]:
cell_area = xr.open_dataset(inputpath_Schmidt + 'cellarea_05degrees.nc')['cell_area']

In [17]:
melt_front = mask_front.copy()
melt_front_per_basin_list = []
for id_basin, bbasin in enumerate(tqdm(ID_list.keys())):  
    cell_area_bb = cell_area.where(mask_front == id_basin+1)
    cell_area_bb_sum = cell_area.where(mask_front == id_basin+1).sum()

    melt_front = melt_front.where(mask_front != id_basin+1, melt_all.sel(basin=bbasin) * cell_area_bb / cell_area_bb_sum).drop('basin').drop('quantile')
    melt_front_basin = (melt_front * np.nan).where(mask_front != id_basin+1, melt_all.sel(basin=bbasin) * cell_area_bb / cell_area_bb_sum).drop('basin').drop('quantile')
    melt_front_per_basin_list.append(melt_front_basin.where(np.isfinite(melt_front_basin), 0).assign_coords({'IMBIE_basin': id_basin+1}))
melt_front_per_basin = xr.concat(melt_front_per_basin_list, dim='IMBIE_basin')


  0%|          | 0/18 [00:00<?, ?it/s]

In [81]:
melt_front_per_basin_norm = melt_front_per_basin/melt_front_per_basin.sum(['lon','lat'])

In [ ]:
melt_front_per_basin_norm_per_m2 = (melt_front_per_basin/cell_area) / melt_front_per_basin.sum(['lon','lat'])

In [153]:
z_file = mask_front.copy()
zmin_list = []
zmax_list = []
for id_basin, bbasin in enumerate(tqdm(ID_list.keys())): 
    zmin_list.append((z_file.where(mask_front != id_basin+1, zmin_all.sel(basin=bbasin))).assign_coords({'IMBIE_basin': id_basin+1}))
    zmax_temp = z_file.where(mask_front != id_basin+1, zmax_all.sel(basin=bbasin))
    zmax_temp = zmax_temp.where(bathy >= zmax_temp, bathy).where(mask_front == id_basin+1)
    zmax_list.append(zmax_temp.assign_coords({'IMBIE_basin': id_basin+1}))
zmin_front_per_basin = xr.concat(zmin_list, dim='IMBIE_basin').drop('basin').drop('quantile')
zmax_front_per_basin = xr.concat(zmax_list, dim='IMBIE_basin').drop('basin').drop('quantile')


  0%|          | 0/18 [00:00<?, ?it/s]

In [150]:
(mask_front != id_basin+1).plot()

In [154]:
zmax_front_per_basin.isel(IMBIE_basin=0).plot()

In [151]:
zmax_temp = z_file.where(mask_front != id_basin+1, zmax_all.sel(basin=bbasin))
zmax_temp = zmax_temp.where(bathy >= zmax_temp, bathy).where(mask_front == id_basin+1)

In [152]:
zmax_temp.plot()

In [49]:
for bb in range(1,19):
    plt.figure()
    melt_front_per_basin.sel(IMBIE_basin=bb).plot()

In [55]:
melt_front_per_basin.sum(['lon','lat'])

<xarray.DataArray 'mask' (IMBIE_basin: 18)>
array([ 57.7,  25.2,  20.3,  51.2,  93.4,  18.2,  16. ,   7.4,   2.1,
       151.7, 305.8,  77.1, 132.1,  17.8,  19.3,  53.5,   8.2,  24.2])
Coordinates:
  * IMBIE_basin  (IMBIE_basin) int64 1 2 3 4 5 6 7 8 ... 11 12 13 14 15 16 17 18

In [40]:
ID_list

{'A-Ap': [1, 2, 3, 4, 5, 6, 7, 8, 9],
 'Ap-B': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20],
 'B-C': [21],
 'C-Cp': [22, 23, 24, 25, 26, 27, 28, 29],
 'Cp-D': [30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40],
 'D-Dp': [41, 42, 43, 44, 45, 46, 47],
 'Dp-E': [48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65],
 'E-Ep': [66],
 'Ep-F': [67, 68, 69, 70, 71, 72, 73, 74, 75, 76],
 'F-G': [77, 78, 79, 80, 81, 82],
 'G-H': [83, 84, 85, 86, 87, 88, 89],
 'H-Hp': [90, 91, 92, 93, 94, 95, 96],
 'Hp-I': [97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111],
 'I-Ipp': [112, 113, 114, 115, 116, 117],
 'Ipp-J': [118, 119, 120, 121, 122, 123],
 'J-Jpp': [124],
 'Jpp-K': [125],
 'K-A': [126, 127, 128, 129, 130, 131, 132, 133]}

In [82]:
# transform it to weights instead of absolute values 
melt_front_norm = melt_front/melt_front.sum()

Distribute over zmin and zmax corresponding to bathymetry

In [83]:
bathy = -1*BedMachine_05['bed']

In [84]:
zmin_front = mask_front.copy()
zmax_front = mask_front.copy()
for id_basin, bbasin in enumerate(tqdm(ID_list.keys())): 
    zmin_front = zmin_front.where(mask_front != id_basin+1, zmin_all.sel(basin=bbasin)).drop('basin').drop('quantile')
    zmax_front = zmax_front.where(mask_front != id_basin+1, zmax_all.sel(basin=bbasin)).drop('basin').drop('quantile')
# cut where bathymetry is shallower
zmax_front = zmax_front.where(bathy >= zmax_front, bathy).where(np.isfinite(mask_front))

  0%|          | 0/18 [00:00<?, ?it/s]

In [158]:
all_info = xr.Dataset()
all_info['melt_weight_per_basin'] = melt_front_per_basin_norm
all_info['melt_weight_per_basin'].attrs['standard_name'] = 'IMBIE basin-wide melt from Davison distributed over the ice-shelf front cells, normalised by cell-area and by basin-wide melt'
all_info['melt_weight_per_basin'].attrs['units'] = 'between 0 and 1'
all_info['melt_weight_per_basin'].attrs['method'] = 'The sum over all Antarctica is 18 as the integral of the weights is 1 for each of the 18 basins.'
#all_info['melt_weight'] = melt_front_norm.where(np.isfinite(melt_front_norm), 0)
#all_info['melt_weight'].attrs['standard_name'] = 'IMBIE basin-wide melt from Davison distributed over the ice-shelf front cells, normalised by cell-area and Antarctic-wide melt'
#all_info['melt_weight'].attrs['units'] = 'between 0 and 1'
#all_info['melt_weight'].attrs['method'] = 'The sum over all Antarctica is 1 as the weights are normalised by the Antarctic-wide melt. However, note that the melt has been redistributed in the same manner as for melt_weight_per_basin'
all_info['z_min_per_basin'] = zmin_front_per_basin.where(np.isfinite(zmin_front_per_basin), 0)
all_info['z_min_per_basin'].attrs['standard_name'] = 'Minimal depth where to inject the sub-ice-shelf meltwater'
all_info['z_min_per_basin'].attrs['units'] = 'm below sea-level'
all_info['z_min_per_basin'].attrs['method'] = 'This is the basin-wide shallowest point of the ice-shelf draft at the ice-shelf front (1st percentile)'
all_info['z_max_per_basin'] = zmax_front_per_basin.where(np.isfinite(zmax_front_per_basin), 0)
all_info['z_max_per_basin'].attrs['standard_name'] = 'Maximal depth where to inject the sub-ice-shelf meltwater'
all_info['z_max_per_basin'].attrs['units'] = 'm below sea-level'
all_info['z_max_per_basin'].attrs['method'] = 'This is either the shallowest point of: (1) basin-wide deepest bathymetry (99th percentile) and (2) bathymetry at the cell of meltwater injection'
#all_info['z_min'] = zmin_front.where(np.isfinite(zmin_front), 0)
#all_info['z_min'].attrs['standard_name'] = 'Minimal depth where to inject the sub-ice-shelf meltwater'
#all_info['z_min'].attrs['units'] = 'm below sea-level'
#all_info['z_min'].attrs['method'] = 'This is the shallowest point of the ice-shelf draft at the ice-shelf front (1st percentile)'
#all_info['z_max'] = zmax_front.where(np.isfinite(zmax_front), 0)
#all_info['z_max'].attrs['standard_name'] = 'Maximal depth where to inject the sub-ice-shelf meltwater'
#all_info['z_max'].attrs['units'] = 'm below sea-level'
#all_info['z_max'].attrs['method'] = 'This is either the shallowest point of: (1) deepest bathymetry in the ice-shelf cavity and (2) deepest bathymetry at the cell of meltwater injection (99th percentile)'

for att in ['long_name','flag_values','flag_meanings','source']:
    del all_info['z_min_per_basin'].attrs[att]
    del all_info['z_max_per_basin'].attrs[att]

In [159]:
all_info.attrs['description'] = 'The melt is based on Davison et al. 2023 and the geometric properties inferred from below the ice-shelf cavities are based on BedMachine3 (Morlighem 2022)'
all_info.attrs['reference'] = 'B.J. Davison, A.E. Hogg, N. Gourmelen, L. Jakob, J. Wuite, T. Nagler, C.A. Greene, J. Andreasen, M.E. Engdahl (2023): Annual mass budget of Antarctic ice shelves from 1997 to 2021. Science Advances, doi:10.1126/sciadv.adi0186'
all_info.attrs['reference2'] = 'Morlighem, M. (2022). MEaSUREs BedMachine Antarctica. (NSIDC-0756, Version 3). [Data Set]. Boulder, Colorado USA. NASA National Snow and Ice Data Center Distributed Active Archive Center. doi: 10.5067/FPSU0V1MWUB6. Date Accessed 05-15-2025.'
all_info.attrs['creation'] = 'C. Burgard on Mon Jun 16 12:52 2025'

In [160]:
all_info.to_netcdf(inputpath_Schmidt + 'subshelf_meltwater_injection_weights_and_depths.nc')

In [ ]:
# if we want the weights in kg/s/m2


Check output

In [89]:
all_info = xr.open_dataset(inputpath_Schmidt + 'subshelf_meltwater_injection_weights_and_depths.nc')

In [95]:
all_info['z_min'].plot(vmax=10)

In [98]:
all_info['z_min'].plot()